In [13]:
""" Third take on machine learning features... 

CURRENTLY TESTING:
(MINIMAL UP-FRONT COMPUTATION)
1. Counts on all intervals/steps (NOT pre-seeding data with the pattern to look outfor
step pattern of 0,3,7 for minor, vs. 0,4,7 for major)
2. Minor mode offers 9 notes vs. majors 7 notes


Consider using Irish folkmusic from Music21 corpus... single stream!
"""

###############################################

import music21
import pandas as pd
import numpy as np
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression

In [17]:
training_data = []

# read test files and construct columns
filenames = open('test_files.txt').read().split('\n')

for filename in filenames:
    
    # Create empty score_instance to hold data of training features + outcome
    score_instance = {}

    # Convert MIDI file to score and iterate over notes in score 
    # Save relevant individual Note attributes into list (incl. Note obj, name, freq, major/minor at given measure)
    score = music21.converter.parse('MIDI test files/Cello solos/' + filename)
    note_attributes = []
    for note in music21.alpha.theoryAnalysis.theoryAnalyzer.getNotes(score, 0):
        if note == None:
            pass
        else:
#             measure = int(note.offset)
#             key = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, measure)
            note_attributes.append([note, note.name, note.frequency])
    print "Note attributes for file {} completed".format(filename)
            
    # Calculate global attributes of score (incl: Avg interval, avg steps, is_major)
    # Note: At the moment, this doesn't take into account changes b/w major + minor w/in a score

    score_instance['note_count'] = {}
    for i in range(len(note_attributes)):
        note_name = note_attributes[i][1]
        score_instance['note_count'][note_name] = score_instance['note_count'].get(note_name, 0) + 1


    score_instance['step_count'] = {}
    for i in range(1, len(note_attributes)):
        interval = music21.interval.Interval(noteStart=note_attributes[i-1][0], noteEnd=note_attributes[i][0])
        step = int((interval.cents)/100)
        score_instance['step_count'][('step' + step)] = score_instance['step_count'].get(step, 0) + 1

            
    mode_at_measure_0 = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, 0).mode
    is_major = (mode_at_measure_0 == 'major')
    score_instance['is_major'] = is_major    
    
    # Saves all global attributes into a dictionary and appends to global set of all training data
    # Used to create df and plt below
    
    print "Global attributes include: {}".format(score_instance)
    training_data.append(score_instance)

Note attributes for file cs1-1pre.mid completed
Global attributes include: {'step_count': {0: 5, 1: 65, 2: 80, 3: 23, 4: 20, 5: 15, 6: 25, 7: 13, 8: 16, 9: 17, 10: 12, 11: 5, 12: 3, 22: 1, -11: 5, -19: 1, -18: 3, -17: 1, -12: 1, -2: 91, -10: 11, -9: 12, -8: 16, -7: 9, -6: 23, -5: 10, -4: 23, -3: 33, -1: 53}, 'note_count': {'A': 81, 'E-': 6, 'C': 68, 'B': 78, 'E': 63, 'D': 80, 'G': 95, 'F': 12, 'C#': 18, 'B-': 3, 'F#': 84, 'G#': 5}, 'is_major': True}
Note attributes for file cs1-2all.mid completed
Global attributes include: {'step_count': {0: 24, 1: 114, 2: 150, 3: 48, 4: 40, 5: 22, 6: 6, 7: 30, 8: 10, 9: 10, 10: 8, 12: 2, 13: 2, 14: 3, 16: 2, 17: 2, 19: 1, 21: 1, 22: 2, -11: 2, -17: 2, -15: 4, -14: 2, -12: 8, -1: 96, -10: 14, -9: 16, -8: 4, -7: 16, -6: 6, -5: 34, -4: 42, -3: 64, -2: 126}, 'note_count': {'A': 148, 'E-': 2, 'C': 100, 'B': 130, 'E': 96, 'D': 138, 'G': 152, 'F': 16, 'C#': 26, 'B-': 4, 'F#': 94, 'G#': 8}, 'is_major': True}
Note attributes for file cs1-3cou.mid completed
Glo

In [74]:
# Create dataframe to hold all relevant training feature and outcome data
# Incl: 'note_count', 'step_count', 'is_major'
df = None

for i in range(len(training_data)):
#     print training_data[i]['note_count']

    note_count_df = pd.DataFrame(training_data[i]['note_count'], index=[0], dtype='float64')
    step_count_df = pd.DataFrame(training_data[i]['step_count'], index=[0], dtype='float64')
    outcome_df = pd.Series(training_data[i]['is_major'], name='is_major')

    instance_df = pd.concat([note_count_df, step_count_df, outcome_df], axis=1)
    if i == 0:
        df = instance_df
    else:
        df = pd.concat([df, instance_df], axis=0, ignore_index=True)



In [71]:
# Plot training data, O = major outcome, x = minor outcome
# is_major_filter = (df['is_major'] == True)
# is_minor_filter = (df['is_major'] == False)

# plt.plot(df[is_major_filter],
#          linestyle='', 
#          marker='o')

# plt.plot(df[is_minor_filter], 
#          linestyle='', 
#          marker='x')

# plt.xlabel('Step')
# plt.ylabel('Step Frequency')

In [107]:
# # Plot training data, O = major outcome, x = minor outcome
# is_major_filter = (df['is_major'] == True)
# is_minor_filter = (df['is_major'] == False)

# plt.plot(df[is_major_filter], 
#          linestyle='', 
#          marker='o')

# plt.plot(df[is_minor_filter], 
#          linestyle='', 
#          marker='x')

# plt.xlabel('Notes')
# plt.ylabel('Note Frequency')

for x in df.columns[:55]:
    print x


-28
-27
-24
-23
-22
-21
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
26
27
28
31


In [ ]:
# NEXT STEPS: 
# 1. Look up how to pull data from dictionaries into pd dataframe
# 2. Explore Patsy, is this necessary intermediate between pds and scikit-learn
# 3. Anything more to declare for classifier and params before running logistic reg

In [133]:
from patsy import dmatrices
# Establishing training data
# X_train = []
# y_train = []

# y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
#                   religious + educ + C(occupation) + C(occupation_husb)',
#                   dta, return_type="dataframe")

# all_columns = " ~ "
# for x in df.columns[:-2]:
#     x = str(x)
#     all_columns = all_columns + x + " + "
# all_columns += df.columns[-2]

# y_column = df.columns[-1]
# all_columns = y_column + all_columns

# print df.columns

#  ,         -23,         -22,
#                -21,         -20,         -19,         -18,         -17,
#                -16,         -15,         -14,         -13,         -12,
#                -11,         -10,          -9,          -8,          -7,
#                 -6,          -5,          -4,          -3,          -2,
#                 -1,           0,           1,           2,           3,
#                  4,           5,           6,           7,           8,
#                  9,          10,          11,          12,          13,
#                 14,          15,          16,          17,          18,
#                 19,          20,          21,          22,          23,
#                 24,          26,          27,          28,          31,
#               u'A',        u'B',       u'B-',        u'C',       u'C#',
#               u'D',        u'E',       u'E-',        u'F',       u'F#',
#               u'G',       u'G#''
            
y_train, X_train = dmatrices('is_major ~ A + B + B', 
                             df, return_type='dataframe')

y_train = np.ravel(y_train)

# for song in training_data[:19]:
#     X_train.append((song['note_count'], song['step_count']))
#     y_train.append(song['is_major'])
    
# # Creating classifiers
lr = LogisticRegression()

# # Fit model to the training data
lr.fit(X_train, y_train)

Index([        -28,         -27,         -24,         -23,         -22,
               -21,         -20,         -19,         -18,         -17,
               -16,         -15,         -14,         -13,         -12,
               -11,         -10,          -9,          -8,          -7,
                -6,          -5,          -4,          -3,          -2,
                -1,           0,           1,           2,           3,
                 4,           5,           6,           7,           8,
                 9,          10,          11,          12,          13,
                14,          15,          16,          17,          18,
                19,          20,          21,          22,          23,
                24,          26,          27,          28,          31,
              u'A',        u'B',       u'B-',        u'C',       u'C#',
              u'D',        u'E',       u'E-',        u'F',       u'F#',
              u'G',       u'G#', u'is_major'],
      dtype='obje

ValueError: Found arrays with inconsistent numbers of samples: [36 72]